In [1]:
from flask import Flask, request, jsonify,render_template
from flasgger import Swagger, LazyString, LazyJSONEncoder, swag_from
from unidecode import unidecode
import re
import pandas as pd
import sqlite3

In [2]:
app = Flask(__name__)

app.json_encoder = LazyJSONEncoder
swagger_template = dict(
info = {
    'title' : LazyString(lambda: 'Binar Academy Gold Challenge'),
    'version' : LazyString(lambda : '1.0.0'),
    'description': LazyString(lambda : 'Membuat API Cleansing Data'),
},
    host = LazyString(lambda: request.host)
)

swagger_config = {
    'headers': [],
    'specs': [
        {
        'endpoint': 'docs',
        'route': '/docs.json',
        }
    ],
    'static_url_path': '/flasgger_static',
    'swagger_ui': True,
    'specs_route': '/docs/'
}
swagger = Swagger(app, template=swagger_template,
                 config = swagger_config)

def get_db_connection():
    conn = sqlite3.connect('data/Challenge.db')
    conn.row_factory = sqlite3.Row
    return

def _remove_punct(s):
    return re.sub(r"[^\w\d\s]+", "",s)

def replace_ascii(s):
    return unidecode

def remove_ascii2(s):
    return re.sub(r"\\x[A-Za-z0-9./]+","", unidecode(s))

def remove_byte(s):
    return bytes(s, 'utf-8').decode('utf-8', 'ignore')

def remove_newlines(s):
    return re.sub(r'\n', ' ',s)

def remove_morespace(s):
    return re.sub('  +', '',s)

def remove_newlines_text(s):
    return re.sub(r'\n.{0}', ' ', s)

@swag_from('C:/Users/User/Binar/Gold Binar Academy/docs/swagger_config_post.yml', methods=['POST'])
@app.route('/', methods=['POST'])
def remove_punct_poct():
    s = request.get_json(force=True)
        # return jsonify({"CleanText":non_punct})

    print(s['text'])
    result = remove_byte(s['text'])
    result = _remove_punct(result)
    conn = sqlite3.connect("data/Challenge.db")
#     conn.execute("insert into Tweet (Dirt_Text, Clean_Text) values (?,?)", (s['text']), result)
    conn.commit()
    conn.close()
    return jsonify({"Clean_Text":result})

@swag_from('C:/Users/User/Binar/Gold Binar Academy/docs/swagger_config_file.yml', methods=['POST'])
@app.route("/get_text/v1", methods=['GET'])
def return_text():
    CleanText_input = request.args.get('CleanText')
    return_text = {
        "text":f"hasil bersih{CleanText_input}"
    }
   
    return jsonify(return_text)

@swag_from('C:/Users/User/Binar/Gold Binar Academy/docs/swagger_config_form.yml', methods=['POST'])
@app.route("/upload", methods=['POST'])
def post_file():
    file = request.files["file"]
    df = pd.read_csv(file, encoding='ISO-8859-1')
    df['CleanTweet'] = df['Tweet'].apply(remove_ascii2)
    df['CleanTweet1'] = df['CleanTweet'].apply(remove_byte)
    df.drop(df.columns[13], axis=1, inplace=True)
    df['CleanTweet2'] = df['CleanTweet1'].apply(remove_newlines)
    df.drop(df.columns[13], axis=1, inplace=True)
    df['CleanTweet3'] = df['CleanTweet2'].apply(remove_newlines_text)
    df.drop(df.columns[13], axis=1, inplace=True)
    df['CleanTweet4'] = df['CleanTweet3'].apply(_remove_punct)
    df.drop(df.columns[13], axis=1, inplace=True)
    df['CleanTweet'] = df['CleanTweet4'].apply(remove_morespace)
    df.drop(df.columns[13], axis=1, inplace=True)
    conn = sqlite3.connect("data/Challenge.db")
    df.to_sql('Challenge_Data', con=conn, index=False, if_exists='append')
    df.to_csv("data_clean.csv", sep = ";")
    conn.close()
    return jsonify({
        "text":"Successfully upload file"
    })
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Dec/2022 17:28:51] "GET / HTTP/1.1" 405 -
127.0.0.1 - - [19/Dec/2022 17:28:52] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Dec/2022 17:28:54] "GET /docs/ HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2022 17:28:54] "GET /flasgger_static/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Dec/2022 17:28:54] "GET /flasgger_static/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Dec/2022 17:28:54] "GET /flasgger_static/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Dec/2022 17:28:54] "GET /flasgger_static/lib/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Dec/2022 17:28:55] "GET /docs.json HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2022 17:29:14] "POST /upload HTTP/1.1" 200 -
